# Examples from TTR overview lecture in `pyttr`

In [ ]:
from ttrtypes import Type,BType,PType,Pred,Possibility,RecType,Fun,\
                     TTRStringType,KPlusStringType,LazyObj,TTRString
from records import Rec
from utils import show, show_latex,print_latex,forall

: 

## Judgement

In [2]:
T = Type()
T.judge('a')

print(T.query('a'))
show_latex(T)

True


<IPython.core.display.Latex object>

## Basic types

In [3]:
Ind = BType('Ind')
Ind.judge('a')

print(Ind.query('a'))
show_latex(Ind)

True


<IPython.core.display.Latex object>

## Witness conditions for types

In [4]:
Real = BType('Real')
Real.learn_witness_condition(lambda n: isinstance(n,float))
print(Real.query(0.5))
print(Real.witness_cache)

True
[0.5]


## Intensionality

In [5]:
T1 = Type()
T2 = Type()
T1.judge('a')
T1.judge('b')
T2.judge('b')
T2.judge('a')

print(T1.witness_cache)
print(set(T1.witness_cache) == set(T2.witness_cache))
print(T1==T2)

['a', 'b']
True
False


## Ptypes

In [6]:
run = Pred('run',[Ind])
hug = Pred('hug',[Ind,Ind])
Ind.judge('b')
Ind.judge('d')
run_d = PType(run,['d'])
hug_b_d = PType(hug,['b','d'])
run_d.judge('e1')
hug_b_d.judge('e2')

print(run_d.query('e1'))
print(hug_b_d.query('e1'))

True
False


In [7]:
show_latex(hug_b_d)

<IPython.core.display.Latex object>

In [8]:
show_latex(run_d)

<IPython.core.display.Latex object>

## Possibilities (models)

In [9]:
M1 = Possibility('M1')

Ind.in_poss(M1)
run_d.in_poss(M1)
hug_b_d.in_poss(M1)

print(show(M1))


M1:
_____________________________________________
Ind: [a, b, d]
run(d): [e1]
hug(b, d): [e2]
_____________________________________________



In [10]:
M2 = Possibility('M2')

BType('Ind',M2).judge('c')
PType(run,['c'],M2).judge('e1')

print(show(M2))


M2:
_____________________________________________
Ind: [c]
run(c): [e1]
_____________________________________________



## Witness conditions associated with ptypes

In [11]:
left = Pred('left',[Ind,Ind])
right = Pred('right',[Ind,Ind])
left.learn_witness_fun(lambda args: PType(right,[args[1],args[0]]))
right.learn_witness_fun(lambda args: PType(left,[args[1],args[0]]))
left_a_b = PType(left,['a','b'])
right_b_a = PType(right,['b','a'])
M = Possibility('M')
right_b_a.in_poss(M).judge('s1')
left_a_b.in_poss(M).judge('s2')
print(show(M))
print(left_a_b.in_poss(M).query('s1'))
print(right_b_a.in_poss(M).query('s2'))
print(show(M))


M:
_____________________________________________
right(b, a): [s1]
left(a, b): [s2]
_____________________________________________

True
True

M:
_____________________________________________
right(b, a): [s1, s2]
left(a, b): [s2, s1]
_____________________________________________



## Record Types

In [12]:
boy = Pred('boy',[Ind])
dog = Pred('dog',[Ind])
a_boy_hugs_a_dog = RecType({'x':Ind,
                            'c_boy':(Fun('v',Ind,PType(boy,['v'])), ['x']),
                            'y':Ind,
                            'c_dog':(Fun('v',Ind,PType(dog,['v'])), ['y']),
                            'e':(Fun('v1',Ind,Fun('v2',Ind, PType(hug,['v1','v2']))), 
                                     ['x','y'])})
print(show(a_boy_hugs_a_dog))
show_latex(a_boy_hugs_a_dog)

{x : Ind, c_boy : (lambda v:Ind . boy(v), [x]), y : Ind, c_dog : (lambda v:Ind . dog(v), [y]), e : (lambda v1:Ind . lambda v2:Ind . hug(v1, v2), [x, y])}


<IPython.core.display.Latex object>

In [13]:
M3 = Possibility('M3')
Ind3 = BType('Ind',M3)
Ind3.judge('a')
Ind3.judge('b')
PType(boy,['a']).in_poss(M3).judge('s1')
PType(dog,['b']).in_poss(M3).judge('s2')
PType(hug,['a','b']).in_poss(M3).judge('s3')

print(show(M3))


M3:
_____________________________________________
Ind: [a, b]
boy(a): [s1]
dog(b): [s2]
hug(a, b): [s3]
_____________________________________________



In [14]:
r1 = Rec({'x':'a','c_boy':'s1','y':'b','c_dog':'s2','e':'s3'})

print(show(r1))
show_latex(r1)

{x = a, c_boy = s1, y = b, c_dog = s2, e = s3}


<IPython.core.display.Latex object>

In [15]:
print(a_boy_hugs_a_dog.in_poss(M3).query(r1))

True


## String types

In [16]:
M4 = Possibility('M4')
pick_up = Pred('pick_up',[Ind,Ind])
attract_attention = Pred('attract_attention',[Ind,Ind])
throw = Pred('throw',[Ind,Ind])
run_after = Pred('run_after',[Ind,Ind])
return_stick = Pred('return_stick',[Ind,Ind,Ind])
Ind4 = BType('Ind')
Ind4.in_poss(M4)
Ind4.judge('a')
Ind4.judge('b')
Ind4.judge('c')
pick_up_a_c = PType(pick_up,['a','c'])
attract_attention_a_b = PType(attract_attention,['a','b'])
throw_a_c = PType(throw,['a','c'])
run_after_b_c = PType(run_after,['b','c'])
pick_up_b_c = PType(pick_up,['b','c'])
return_stick_b_c_a = PType(return_stick,['b','c','a'])

FetchOnce_a_b_c = TTRStringType([pick_up_a_c,attract_attention_a_b,throw_a_c,run_after_b_c,
                                 pick_up_b_c,return_stick_b_c_a])
Fetch_a_b_c = KPlusStringType(FetchOnce_a_b_c)
print(show(Fetch_a_b_c))
show_latex(Fetch_a_b_c)

(pick_up(a, c)^attract_attention(a, b)^throw(a, c)^run_after(b, c)^pick_up(b, c)^return_stick(b, c, a))+


<IPython.core.display.Latex object>

In [17]:
pick_up_a_c.in_poss(M4).judge('e1')
attract_attention_a_b.in_poss(M4).judge('e2')
throw_a_c.in_poss(M4).judge('e3')
run_after_b_c.in_poss(M4).judge('e4')
pick_up_b_c.in_poss(M4).judge('e5')
return_stick_b_c_a.in_poss(M4).judge('e6')
e1_6 = TTRString(['e1','e2','e3','e4','e5','e6'])
print(show(e1_6))
show_latex(e1_6)

"e1 e2 e3 e4 e5 e6"


<IPython.core.display.Latex object>

In [18]:
print(FetchOnce_a_b_c.in_poss(M4).query(e1_6))
print(Fetch_a_b_c.in_poss(M4).query(e1_6))

True
True


In [19]:
human = Pred('human',[Ind])
stick = Pred('stick',[Ind])
play_fetch = Pred('play_fetch',[Ind,Ind])
T = RecType({'x':Ind,
             'c_human':(Fun('v',Ind,PType(human,['v'])),['x']),
             'y':Ind,
             'c_dog':(Fun('v',Ind,PType(dog,['v'])),['y']),
             'z':Ind,
             'c_stick':(Fun('v',Ind,PType(stick,['v'])),['z']),
             'e':(Fun('v1',Ind,Fun('v2',Ind,Fun('v3',Ind, 
                                                TTRStringType([PType(pick_up,['v1','v3']),
                                                               PType(attract_attention,['v1','v2'])])))),
                 ['x','y','z'])})
f = Fun('r',T,RecType({'e':PType(play_fetch,[LazyObj(['r','.','x']),LazyObj(['r','.','y'])])}))

print(show(f))
show_latex(f)

lambda r:{x : Ind, c_human : (lambda v:Ind . human(v), [x]), y : Ind, c_dog : (lambda v:Ind . dog(v), [y]), z : Ind, c_stick : (lambda v:Ind . stick(v), [z]), e : (lambda v1:Ind . lambda v2:Ind . lambda v3:Ind . pick_up(v1, v3)^attract_attention(v1, v2), [x, y, z])} . {e : play_fetch([r, ., x], [r, ., y])}


<IPython.core.display.Latex object>